In [20]:
import streamlit as st
import pandas as pd
import requests
import plotly.express as px

# ------------------ PAGE CONFIG ------------------
st.set_page_config(
    page_title="FitPulse – Health Dashboard",
    page_icon="📊",
    layout="wide"
)

# ------------------ STYLES ------------------
st.markdown("""
<style>
.block {
    background-color: #f4f6f9;
    padding: 20px;
    border-radius: 12px;
}
.metric-box {
    background-color: white;
    padding: 15px;
    border-radius: 10px;
    text-align: center;
}
</style>
""", unsafe_allow_html=True)

# ------------------ SIDEBAR ------------------
with st.sidebar:
    st.header("⚙️ Configuration")
    backend_url = st.text_input(
        "FastAPI Backend URL",
        value="https://commonsensibly-uncompanionable-anton.ngrok-free.dev/"
    )
    st.markdown("---")
    st.info("Upload a CSV file to analyze health metrics")

# ------------------ HEADER ------------------
st.title("📊 FitPulse – Health Data Dashboard")
st.caption("Upload CSV → Backend cleans data → Interactive insights")

# ------------------ FILE UPLOAD ------------------
uploaded_file = st.file_uploader("📂 Upload CSV File", type=["csv"])

# ------------------ PROCESS DATA ------------------
if uploaded_file and backend_url:

    with st.spinner("🔄 Processing data..."):
        try:
            response = requests.post(
                f"{backend_url.rstrip('/')}/preprocess",
                files={"file": uploaded_file}
            )

            if response.status_code != 200:
                st.error("❌ Backend Error")
                st.write(response.text)
                st.stop()

            result = response.json()
            df = pd.DataFrame(result["data"])
            status = result["status"]

        except Exception as e:
            st.error(f"Connection failed: {e}")
            st.stop()

    st.success(status)

    # Convert timestamp
    if "timestamp" in df.columns:
        df["timestamp"] = pd.to_datetime(df["timestamp"])

    numeric_cols = df.select_dtypes(include="number").columns.tolist()

    # ------------------ TABS ------------------
    tab1, tab2, tab3 = st.tabs(["📄 Data Preview", "📈 Visualizations", "🧾 Summary"])

    # ================= TAB 1 =================
    with tab1:
        st.subheader("Processed Dataset (Preview)")
        st.dataframe(df.head(500), use_container_width=True)
        st.caption("Showing first 500 rows for performance")

    # ================= TAB 2 =================
    with tab2:
        st.subheader("Health Visualizations")

        if not numeric_cols:
            st.warning("No numeric columns available for visualization.")
        else:
            col1, col2 = st.columns(2)

            # HEART RATE
            heart_cols = [c for c in numeric_cols if "heart" in c]
            if heart_cols and "timestamp" in df.columns:
                with col1:
                    fig = px.line(
                        df,
                        x="timestamp",
                        y=heart_cols[0],
                        title="❤️ Heart Rate Over Time"
                    )
                    st.plotly_chart(fig, use_container_width=True)

            # STEPS
            step_cols = [c for c in numeric_cols if "step" in c]
            if step_cols and "timestamp" in df.columns:
                with col2:
                    fig = px.bar(
                        df,
                        x="timestamp",
                        y=step_cols[0],
                        title="🚶 Steps Over Time"
                    )
                    st.plotly_chart(fig, use_container_width=True)

            # GENERIC METRIC
            other_metrics = [
                c for c in numeric_cols
                if c not in heart_cols + step_cols
            ]
            if other_metrics and "timestamp" in df.columns:
                fig = px.line(
                    df,
                    x="timestamp",
                    y=other_metrics[0],
                    title=f"📈 {other_metrics[0].replace('_',' ').title()}"
                )
                st.plotly_chart(fig, use_container_width=True)

    # ================= TAB 3 =================
    with tab3:
        st.subheader("Health Summary")

        col1, col2, col3, col4 = st.columns(4)

        if heart_cols:
            col1.metric(
                "❤️ Avg Heart Rate",
                f"{df[heart_cols[0]].mean():.1f} bpm"
            )

        if step_cols:
            col2.metric(
                "🚶 Total Steps",
                f"{int(df[step_cols[0]].sum())}"
            )

        sleep_cols = [c for c in numeric_cols if "sleep" in c or "duration" in c]
        if sleep_cols:
            col3.metric(
                "😴 Avg Sleep",
                f"{df[sleep_cols[0]].mean():.2f} hrs"
            )

        if "timestamp" in df.columns:
            days = (df["timestamp"].max() - df["timestamp"].min()).days + 1
            col4.metric("📅 Days Covered", days)

        st.markdown("---")

        st.subheader("Download Cleaned Data")
        csv = df.to_csv(index=False).encode("utf-8")
        st.download_button(
            "⬇️ Download CSV",
            csv,
            file_name="processed_health_data.csv",
            mime="text/csv"
        )

else:
    st.info("👆 Upload a CSV file to begin")


2026-01-13 02:23:29.635 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-13 02:23:29.636 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-13 02:23:29.636 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-13 02:23:29.638 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-13 02:23:29.639 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-13 02:23:29.640 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-13 02:23:29.641 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-13 02:23:29.644 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
# 1. Force install localtunnel globally (Fixes the "Ok to proceed?" error)
!npm install -g localtunnel

# 2. Install Python libraries
!pip install -q streamlit plotly requests

# 3. Get the Tunnel Password
import urllib
print("🔐 Password for Tunnel:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

# 4. Launch Streamlit
# Note: We use 'lt' command now instead of 'npx'
print("🔗 Click the link below when it appears:")
!streamlit run frontend.py & lt --port 8501

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧
changed 22 packages in 2s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧🔐 Password for Tunnel: 34.186.19.66
🔗 Click the link below when it appears:
your url is: https://cruel-ways-look.loca.lt



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.186.19.66:8501

2026-01-13 02:24:42.890 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='content'`.
2026-01-13 02:24:43.309 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='content'`.
2026-01-13 02:24:43.445 Please replace `use_container_width` with `width`.

`use_container_width` will 